In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import numpy as np
import os
import pandas as pd
import sys
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

2.5.1+cu121
True
NVIDIA GeForce RTX 2070
cuda


In [3]:
# # csv_path = '/content/drive/MyDrive/rupa/archive/labeled_anomalies.csv'
# csv_path = r'C:\Users\skhandaker\OneDrive - Oklahoma City University\Documents\Anomaly Detection Paper\rupa\archive\labeled_anomalies.csv'

# df = pd.read_csv(csv_path)

# print(df["spacecraft"].unique())

# m_files = [f"{chan_id}.npy" for chan_id in df.loc[df['spacecraft']=='MSL', 'chan_id']]

# print(m_files)

In [4]:
# test_dir = '/content/drive/MyDrive/rupa/archive/data/data/test'
# test_dir = r'C:\Users\skhandaker\OneDrive - Oklahoma City University\Documents\Anomaly Detection Paper\rupa\archive\data\data\test'
test_dir = r"C:\Users\skhandaker\Documents\RANSynCoders-main\RANSynCoders-main\data\test.csv"

m_files = ["test.csv"]

df = pd.read_csv(test_dir)

X = df.values  

print("Shape (rows, columns):", X.shape)

Shape (rows, columns): (87841, 26)


In [5]:
# test_dir = r'C:\Users\skhandaker\OneDrive - Oklahoma City University\Documents\Anomaly Detection Paper\rupa\archive\data\data\test'
test_dir = r"C:\Users\skhandaker\Documents\RANSynCoders-main\RANSynCoders-main\data"

data_list = []

for fname in m_files:
    file_path = os.path.join(test_dir, fname)
    df = pd.read_csv(file_path)
    arr = df.values
    chan_id = fname[:-4]
    chan_ids = np.full((arr.shape[0], 1), chan_id)
    row_numbers = np.arange(arr.shape[0]).reshape(-1, 1)
    combined = np.hstack([arr, chan_ids, row_numbers])

    data_list.append(combined)

X = np.vstack(data_list)

print(X)

[['132480.0' '0.7753739064836348' '0.9091851286139272' ...
  '0.1059907834101382' 'test' '0']
 ['132481.0' '0.7754228365977105' '0.9091417766193804' ...
  '0.1105990783410138' 'test' '1']
 ['132482.0' '0.7754575209491628' '0.9090036199283068' ...
  '0.1152073732718894' 'test' '2']
 ...
 ['220318.0' '0.8646437313877637' '0.636236140496108' ...
  '0.1290322580645161' 'test' '87838']
 ['220319.0' '0.8645532389561273' '0.6434993402480025' ...
  '0.0599078341013824' 'test' '87839']
 ['220320.0' '0.8646953781925993' '0.6453373206525446' ...
  '0.0506912442396313' 'test' '87840']]


In [6]:
print(X.shape)

(87841, 28)


In [7]:
# all_rows = []

# for f in m_files:
#     data = np.load(os.path.join(test_dir, f))
#     all_rows.append(data)

# combined = np.concatenate(all_rows, axis=0).astype(np.float32)


df = pd.DataFrame(X[:, :26], dtype=np.float32)
unique_embeddings = df.drop_duplicates()

print("Unique embeddings:", len(unique_embeddings))
print("Total time steps:", len(df))

print(type(df))
print(type(unique_embeddings))

Unique embeddings: 87841
Total time steps: 87841
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
print(unique_embeddings)

             0         1         2         3         4         5         6   \
0      132480.0  0.775374  0.909185  0.606704  0.660626  0.449968  0.426717   
1      132481.0  0.775423  0.909142  0.607796  0.660655  0.487813  0.442545   
2      132482.0  0.775458  0.909004  0.607988  0.660623  0.469532  0.433682   
3      132483.0  0.775513  0.909200  0.607218  0.660640  0.459910  0.435581   
4      132484.0  0.775587  0.909318  0.606654  0.660643  0.462155  0.436214   
...         ...       ...       ...       ...       ...       ...       ...   
87836  220316.0  0.864548  0.660041  0.590923  0.413322  0.364657  0.380184   
87837  220317.0  0.864483  0.536428  0.590742  0.413331  0.363695  0.378601   
87838  220318.0  0.864644  0.636236  0.590451  0.413619  0.348942  0.377018   
87839  220319.0  0.864553  0.643499  0.590222  0.413689  0.355997  0.379550   
87840  220320.0  0.864695  0.645337  0.590731  0.413732  0.357280  0.371637   

             7         8         9   ...        16 

In [9]:
print(df)

             0         1         2         3         4         5         6   \
0      132480.0  0.775374  0.909185  0.606704  0.660626  0.449968  0.426717   
1      132481.0  0.775423  0.909142  0.607796  0.660655  0.487813  0.442545   
2      132482.0  0.775458  0.909004  0.607988  0.660623  0.469532  0.433682   
3      132483.0  0.775513  0.909200  0.607218  0.660640  0.459910  0.435581   
4      132484.0  0.775587  0.909318  0.606654  0.660643  0.462155  0.436214   
...         ...       ...       ...       ...       ...       ...       ...   
87836  220316.0  0.864548  0.660041  0.590923  0.413322  0.364657  0.380184   
87837  220317.0  0.864483  0.536428  0.590742  0.413331  0.363695  0.378601   
87838  220318.0  0.864644  0.636236  0.590451  0.413619  0.348942  0.377018   
87839  220319.0  0.864553  0.643499  0.590222  0.413689  0.355997  0.379550   
87840  220320.0  0.864695  0.645337  0.590731  0.413732  0.357280  0.371637   

             7         8         9   ...        16 

In [10]:
GPT_CONFIG = {
  "vocab_size": 88000, # Vocabulary size
  "context_length": 64, # Context length
  "emb_dim": 26, # Embedding dimension
  "n_heads": 2, # Number of attention heads
  "n_layers": 12, # Number of layers
  "drop_rate": 0.1, # Dropout rate
  "qkv_bias": False # Query-Key-Value bias
}

In [11]:
class MultiHeadAttention(nn.Module):
 def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
  super().__init__()
  assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
  self.d_out = d_out
  self.num_heads = num_heads
  self.head_dim = d_out // num_heads
  self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
  self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
  self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
  self.out_proj = nn.Linear(d_out, d_out)
  self.dropout = nn.Dropout(dropout)
  self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
 def forward(self, x):
  b, num_tokens, d_in = x.shape
  keys = self.W_key(x)
  queries = self.W_query(x)
  values = self.W_value(x)
  keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
  values = values.view(b, num_tokens, self.num_heads, self.head_dim)
  queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
  keys = keys.transpose(1, 2)
  queries = queries.transpose(1, 2)
  values = values.transpose(1, 2)
  attn_scores = queries @ keys.transpose(2, 3)
  mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

  attn_scores.masked_fill_(mask_bool, -torch.inf)
  attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
  attn_weights = self.dropout(attn_weights)
  context_vec = (attn_weights @ values).transpose(1, 2)

  context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
  context_vec = self.out_proj(context_vec)
  return context_vec

In [12]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

In [13]:
class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), GELU(), nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),)

  def forward(self, x):
    return self.layers(x)

In [14]:
class LayerNorm(nn.Module):
  def __init__(self, emb_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emb_dim))
    self.shift = nn.Parameter(torch.zeros(emb_dim))
  def forward(self, x):
    mean = x.mean(dim=-1, keepdim=True)
    var = x.var(dim=-1, keepdim=True, unbiased=False)
    norm_x = (x - mean) / torch.sqrt(var + self.eps)
    return self.scale * norm_x + self.shift

In [15]:
class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(d_in=cfg["emb_dim"], d_out=cfg["emb_dim"], context_length=cfg["context_length"], num_heads=cfg["n_heads"], dropout=cfg["drop_rate"], qkv_bias=cfg["qkv_bias"])
    self.ff = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
  def forward(self, x):
    shortcut = x
    x = self.norm1(x)
    x = self.att(x)
    x = self.drop_shortcut(x)
    x = x + shortcut
    shortcut = x
    x = self.norm2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut
    return x

In [16]:
c_unique_embeddings = unique_embeddings.values
emb_tuples = [tuple(row) for row in c_unique_embeddings]
emb2idx = {emb: idx for idx, emb in enumerate(emb_tuples)}
def get_embedding_ids(batch):
    batch_np = batch.detach().cpu().numpy().astype(np.float32)

    B, L, D = batch_np.shape
    ids = np.full((B, L), -1, dtype=int)

    for i in range(B):
        for j in range(L):
            ids[i, j] = emb2idx.get(tuple(batch_np[i, j]), -1)

    return ids

In [17]:
class GPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    # self.cfg = cfg
    self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    self.drop_emb = nn.Dropout(cfg["drop_rate"])

    self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

    self.final_norm = LayerNorm(cfg["emb_dim"])
    self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

  def forward(self, in_idx):
    global device
    batch_size, seq_len, _ = in_idx.shape
    in_idx = get_embedding_ids(in_idx)

    in_idx = torch.from_numpy(in_idx).long().to(device)

    tok_embeds = self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
    x = tok_embeds + pos_embeds
    x = self.drop_emb(x)
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits

In [18]:
train_ratio = 0.90
split_idx = int(train_ratio * len(X))
train_data = X[:split_idx]
val_data = X[split_idx:]

In [19]:
print(type(train_data))
print(type(val_data))
print(train_data)
print(val_data)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[['132480.0' '0.7753739064836348' '0.9091851286139272' ...
  '0.1059907834101382' 'test' '0']
 ['132481.0' '0.7754228365977105' '0.9091417766193804' ...
  '0.1105990783410138' 'test' '1']
 ['132482.0' '0.7754575209491628' '0.9090036199283068' ...
  '0.1152073732718894' 'test' '2']
 ...
 ['211533.0' '0.7964793491824964' '0.7946117412252641' ...
  '0.0921658986175115' 'test' '79053']
 ['211534.0' '0.7962846988810984' '0.794414408949061' ...
  '0.0829493087557603' 'test' '79054']
 ['211535.0' '0.7963990709601704' '0.7943934229912035' ...
  '0.0691244239631336' 'test' '79055']]
[['211536.0' '0.7962013152170163' '0.7943621309437571' ...
  '0.0783410138248848' 'test' '79056']
 ['211537.0' '0.7961151965973118' '0.7942946680256983' ...
  '0.0691244239631336' 'test' '79057']
 ['211538.0' '0.7960242269961394' '0.7942100714447726' ...
  '0.0967741935483871' 'test' '79058']
 ...
 ['220318.0' '0.8646437313877637' '0.636236140496108' ...
  '0.129032258

In [20]:
class GPTDatasetV1(Dataset):
  def __init__(self, txt, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    # for txt in txts:
    for i in range(0, len(txt) - max_length, stride):
      input_chunk = txt[i:i + max_length]
      target_chunk = txt[i + 1: i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [21]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
  dataset = GPTDatasetV1(txt, max_length, stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
  return dataloader

In [22]:
train_numeric = train_data[:, :26].astype(np.float32)
val_numeric = val_data[:, :26].astype(np.float32)

train_loader = create_dataloader_v1(
  train_numeric,
  batch_size=256,
  max_length=GPT_CONFIG["context_length"],
  stride=GPT_CONFIG["context_length"],
  drop_last=True,
  shuffle=True,
  num_workers=0
)

val_loader = create_dataloader_v1(
  val_numeric,
  batch_size=256,
  max_length=GPT_CONFIG["context_length"],
  stride=GPT_CONFIG["context_length"],
  drop_last=False,
  shuffle=False,
  num_workers=0
)

In [23]:
def calc_loss_batch(input_batch, target_emb_batch, model, device):
    input_batch = input_batch.to(device)
    target_emb_batch = target_emb_batch.to(device)
    logits = model(input_batch)
    target_ids = get_embedding_ids(target_emb_batch)
    target_ids = torch.from_numpy(target_ids).long().to(device)
    B, L, V = logits.size()
    logits_flat  = logits.view(B * L, V)
    targets_flat = target_ids.view(B * L)
    loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
    return loss

In [24]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
  total_loss = 0
  if len(data_loader) == 0:
    return float("nan")
  elif num_batches is None:
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))

  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      total_loss += loss.item()
    else:
      break

  return total_loss / num_batches

In [25]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
  model.eval()
  with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
  model.train()
  return train_loss, val_loss

In [26]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, eval_freq, eval_iter):
  train_losses, val_losses, track_tokens_seen = [], [], []
  tokens_seen, global_step = 0, -1
  for epoch in range(num_epochs):
    model.train()
    for input_batch, target_batch in train_loader:
      optimizer.zero_grad()
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      loss.backward()
      optimizer.step()
      tokens_seen += input_batch.numel()
      global_step += 1

      if global_step % eval_freq == 0:
        train_loss, val_loss = evaluate_model(model, train_loader, val_loader, device, eval_iter)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        track_tokens_seen.append(tokens_seen)
        print(f"Ep {epoch+1} (Step {global_step:06d}): Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
  return train_losses, val_losses, track_tokens_seen

In [27]:
torch.manual_seed(42)
model = GPTModel(GPT_CONFIG)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(
  model.parameters(),
  lr=0.0004, weight_decay=0.1
)
num_epochs = 100
train_losses, val_losses, tokens_seen = train_model_simple(
  model, train_loader, val_loader, optimizer, device,
  num_epochs=num_epochs, eval_freq=5, eval_iter=1
)

Ep 1 (Step 000000): Train loss 11.546, Val loss 11.550
Ep 2 (Step 000005): Train loss 11.528, Val loss 11.556
Ep 3 (Step 000010): Train loss 11.496, Val loss 11.562
Ep 4 (Step 000015): Train loss 11.478, Val loss 11.566
Ep 6 (Step 000020): Train loss 11.460, Val loss 11.570
Ep 7 (Step 000025): Train loss 11.432, Val loss 11.573
Ep 8 (Step 000030): Train loss 11.417, Val loss 11.579
Ep 9 (Step 000035): Train loss 11.384, Val loss 11.584
Ep 11 (Step 000040): Train loss 11.363, Val loss 11.586
Ep 12 (Step 000045): Train loss 11.347, Val loss 11.588
Ep 13 (Step 000050): Train loss 11.338, Val loss 11.588
Ep 14 (Step 000055): Train loss 11.300, Val loss 11.588
Ep 16 (Step 000060): Train loss 11.288, Val loss 11.593
Ep 17 (Step 000065): Train loss 11.274, Val loss 11.595
Ep 18 (Step 000070): Train loss 11.252, Val loss 11.597
Ep 19 (Step 000075): Train loss 11.227, Val loss 11.598
Ep 21 (Step 000080): Train loss 11.213, Val loss 11.603
Ep 22 (Step 000085): Train loss 11.188, Val loss 11.604


In [28]:
import ast
csv_path = r'C:\Users\skhandaker\OneDrive - Oklahoma City University\Documents\Anomaly Detection Paper\rupa\archive\labeled_anomalies.csv'
df = pd.read_csv(csv_path)

anomaly_masks = {}

for index, row in df.iterrows():
    print(index)
    print(row)
    chan = row["chan_id"]
    num_values = row["num_values"]
    intervals = ast.literal_eval(row["anomaly_sequences"])

    mask = np.zeros(num_values, dtype=bool)

    for start, end in intervals:
        mask[start:end+1] = True

    anomaly_masks[chan] = mask

0
chan_id                                                     P-1
spacecraft                                                 SMAP
anomaly_sequences    [[2149, 2349], [4536, 4844], [3539, 3779]]
class                      [contextual, contextual, contextual]
num_values                                                 8505
Name: 0, dtype: object
1
chan_id                         S-1
spacecraft                     SMAP
anomaly_sequences    [[5300, 5747]]
class                       [point]
num_values                     7331
Name: 1, dtype: object
2
chan_id                                       E-1
spacecraft                                   SMAP
anomaly_sequences    [[5000, 5030], [5610, 6086]]
class                    [contextual, contextual]
num_values                                   8516
Name: 2, dtype: object
3
chan_id                         E-2
spacecraft                     SMAP
anomaly_sequences    [[5598, 6995]]
class                       [point]
num_values                   

In [29]:
context = GPT_CONFIG["context_length"]
forecast_horizon = context

model.eval()
dists, uncertainties, expecteds, y_targets = [], [], [], []

with torch.no_grad():
    for t in range(len(val_data) - context - forecast_horizon):
    # for t in range(5):
        init_seq = val_data[t : t+context, :26].astype(np.float32)
        init_seq = init_seq[np.newaxis]
        init_seq = torch.tensor(init_seq, device=device)
        true_seq = val_data[t+context : t+context+forecast_horizon, :26].astype(np.float32)

        # print(pred_emb)
        predicted_embs = []
        for _ in range(forecast_horizon):
            logits = model(init_seq)
            next_id = logits[0, -1].argmax().item()
            # print(next_id)
            pred_emb = emb_tuples[next_id]
            # print(pred_emb)
            predicted_embs.append(pred_emb)

            next_emb = torch.tensor(pred_emb, device=device)
            next_emb = next_emb.unsqueeze(0).unsqueeze(0)
            init_seq = torch.cat([init_seq, next_emb], dim=1)[:, -context:, :]
        dist = np.linalg.norm(predicted_embs - true_seq)
        
        chan_id = val_data[t, 26]
        row_indices = val_data[t+context : t+context+forecast_horizon, 27].astype(int)
        mask = anomaly_masks[chan_id]
        window_is_anomaly = mask[row_indices].any()
        
        # print(dist)
        # print(window_is_anomaly)
        dists.append(dist)
        y_targets.append(window_is_anomaly)
        # break
            
const = np.sqrt(2.0 / np.pi)

for m in model.modules():
    if isinstance(m, nn.Dropout):
        m.train()

with torch.no_grad():
    for t in range(len(val_data) - context - forecast_horizon):
    # for t in range(5):
        init_seq = val_data[t : t+context, :26].astype(np.float32)
        init_seq = init_seq[np.newaxis]
        init_seq = torch.tensor(init_seq, device=device)
        true_seq = val_data[t+context : t+context+forecast_horizon, :26].astype(np.float32)

        # print(pred_emb)
        predicted_embs = []
        for _ in range(forecast_horizon):
        # for _ in range(2):
            logits = model(init_seq)
            next_id = logits[0, -1].argmax().item()
            pred_emb = torch.tensor(emb_tuples[next_id], device=device, dtype=torch.float32)
            # print(len(pred_emb))
            predicted_embs.append(pred_emb)
        # print(predicted_embs)
        predicted_embs = torch.vstack(predicted_embs)
        uncertainty = predicted_embs.std(0).norm()
        uncertainties.append(uncertainty)
        expecteds.append(const * uncertainty)
        # break

# dists should be Python floats – move them to same device
d_tensor = torch.tensor(dists, device=device, dtype=torch.float32)

# uncertainties and expecteds are lists of CUDA tensors → stack them
u_tensor = torch.tensor(uncertainties, device=device, dtype=torch.float32)
e_tensor = torch.tensor(expecteds, device=device, dtype=torch.float32)

d_norm = (d_tensor - d_tensor.min()) / (d_tensor.max() - d_tensor.min())
u_norm = (u_tensor - u_tensor.min()) / (u_tensor.max() - u_tensor.min())
e_norm = (e_tensor - e_tensor.min()) / (e_tensor.max() - e_tensor.min())

y_preds = d_norm + u_norm + e_norm
y_preds = (y_preds > 1.5).to(torch.int)
y_preds = y_preds.detach().cpu().numpy()
print(y_preds)

KeyError: np.str_('test')

In [ ]:
from sklearn import metrics
import copy

##### Evaluation Metrics #####

def pak(preds, targets, k=20):

    predicts = preds
    actuals = targets

    one_start_idx = np.where(np.diff(actuals, prepend=0) == 1)[0]
    zero_start_idx = np.where(np.diff(actuals, prepend=0) == -1)[0]

    assert len(one_start_idx) == len(zero_start_idx) + 1 or len(one_start_idx) == len(zero_start_idx)

    if len(one_start_idx) == len(zero_start_idx) + 1:
        zero_start_idx = np.append(zero_start_idx, len(predicts))

    for i in range(len(one_start_idx)):
        if predicts[one_start_idx[i]:zero_start_idx[i]].sum() > k / 100 * (zero_start_idx[i] - one_start_idx[i]):
            predicts[one_start_idx[i]:zero_start_idx[i]] = 1

    return predicts

def get_events(y_test, outlier=1, normal=0):
    events = dict()
    label_prev = normal
    event = 0 
    event_start = 0
    for tim, label in enumerate(y_test):
        if label == outlier:
            if label_prev == normal:
                event += 1
                event_start = tim
        else:
            if label_prev == outlier:
                event_end = tim - 1
                events[event] = (event_start, event_end)
        label_prev = label

    if label_prev == outlier:
        event_end = tim - 1
        events[event] = (event_start, event_end)
    return events

def get_composite_fscore_raw(pred_labels, true_events, y_test, return_prec_rec=False):
    epsilon = 1e-8
    tp = np.sum([pred_labels[start:end + 1].any() for start, end in true_events.values()])
    fn = len(true_events) - tp
    rec_e = tp/(tp + fn)
    prec_t = metrics.precision_score(y_test, pred_labels, zero_division=0)
    fscore_c = 2 * rec_e * prec_t / (rec_e + prec_t + epsilon)
    if prec_t == 0 and rec_e == 0:
        fscore_c = 0
    if return_prec_rec:
        return prec_t, rec_e, fscore_c
    return fscore_c

def constant_bias_fn(inputs):

    return np.sum(inputs) / inputs.shape[0]

def improved_cardinality_fn(cardinality, gt_length):
    
    return ((gt_length - 1) / gt_length) ** (cardinality - 1)

def compute_window_indices(binary_labels):
    
    boundaries = np.empty_like(binary_labels)
    boundaries[0] = 0
    boundaries[1:] = binary_labels[:-1]
    boundaries *= -1
    boundaries += binary_labels

    indices = np.nonzero(boundaries)[0].tolist()
    if len(indices) % 2 != 0:
        indices.append(binary_labels.shape[0])
    indices = [(indices[i], indices[i + 1]) for i in range(0, len(indices), 2)]

    return indices

def _compute_overlap(preds, pred_indices, gt_indices, alpha, bias_fn, cardinality_fn, use_window_weight = False):
    n_gt_windows = len(gt_indices)
    n_pred_windows = len(pred_indices)
    total_score = 0.0
    total_gt_points = 0

    i = j = 0
    while i < n_gt_windows and j < n_pred_windows:
        gt_start, gt_end = gt_indices[i]
        window_length = gt_end - gt_start
        total_gt_points += window_length
        i += 1

        cardinality = 0
        while j < n_pred_windows and pred_indices[j][1] <= gt_start:
            j += 1
        while j < n_pred_windows and pred_indices[j][0] < gt_end:
            j += 1
            cardinality += 1

        if cardinality == 0:
            continue
        
        j -= 1

        cardinality_multiplier = cardinality_fn(cardinality, window_length)

        prediction_inside_ground_truth = preds[gt_start:gt_end]
        omega = bias_fn(prediction_inside_ground_truth)

        weight = window_length if use_window_weight else 1

        total_score += alpha * weight
        total_score += (1 - alpha) * cardinality_multiplier * omega * weight

    denom = total_gt_points if use_window_weight else n_gt_windows

    return total_score / denom

def ts_precision_and_recall(anomalies, predictions, alpha = 0,
                            recall_bias_fn = constant_bias_fn,
                            recall_cardinality_fn = improved_cardinality_fn,
                            precision_bias_fn = None,
                            precision_cardinality_fn = None,
                            anomaly_ranges = None,
                            prediction_ranges = None,
                            weighted_precision = False):
    has_anomalies = np.any(anomalies > 0)
    has_predictions = np.any(predictions > 0)

    if not has_predictions and not has_anomalies:
        return 1, 1
    elif not has_predictions or not has_anomalies:
        return 0, 0

    if precision_bias_fn is None:
        precision_bias_fn = recall_bias_fn
    if precision_cardinality_fn is None:
        precision_cardinality_fn = recall_cardinality_fn

    if anomaly_ranges is None:
        anomaly_ranges = compute_window_indices(anomalies)
    if prediction_ranges is None:
        prediction_ranges = compute_window_indices(predictions)

    recall = _compute_overlap(predictions, prediction_ranges, anomaly_ranges, alpha, recall_bias_fn,
                              recall_cardinality_fn)
    precision = _compute_overlap(anomalies, anomaly_ranges, prediction_ranges, 0, precision_bias_fn,
                                 precision_cardinality_fn, use_window_weight=weighted_precision)

    return precision, recall

class ano_evaluator:
    def __init__(self, preds, targets = None):
        assert len(preds) == len(targets)
        
        try:
            preds = np.asarray(preds)
            targets = np.asarray(targets)
        except TypeError:
            preds = np.asarray(preds.cpu())
            targets = np.asarray(targets.cpu())
            
        self.targets = targets
        self.preds = preds
        
    def eval_naive_f1(self):
        f1 = metrics.f1_score(self.targets, self.preds, zero_division = 0)
        prec = metrics.precision_score(self.targets, self.preds, zero_division=0)
        recall = metrics.recall_score(self.targets, self.preds, zero_division=0)
        return f1, prec, recall
        
    def eval_pak_auc(self):
        pak_metrics_list = []
        for k in np.arange(0,110, 10):
            preds_new = copy.deepcopy(self.preds)
            targets_new = copy.deepcopy(self.targets)
            pa_scores = pak(preds_new, targets_new, k)
            pak_metrics_list.append([metrics.f1_score(targets_new, pa_scores, zero_division = 0),
                                     metrics.precision_score(targets_new, pa_scores, zero_division=0),
                                     metrics.recall_score(targets_new, pa_scores, zero_division=0),
                                     ])
        pak_metrics = np.array(pak_metrics_list)
        f1_pak_auc = metrics.auc(np.arange(0,110, 10), pak_metrics[:,0]) / 100.0
        prec_pak_auc = metrics.auc(np.arange(0,110, 10), pak_metrics[:,1]) / 100.0
        recall_pak_auc = metrics.auc(np.arange(0,110, 10), pak_metrics[:,2]) / 100.0

        return f1_pak_auc, prec_pak_auc, recall_pak_auc
    
    def eval_f1_composite(self):
        true_events = get_events(self.targets)
        prec_pw, rec_ew, f1_comp = get_composite_fscore_raw(self.preds, true_events, self.targets, return_prec_rec=True)
        return f1_comp, prec_pw, rec_ew
    
    def eval_f1_range(self):
        epsilon = 1e-8
        label_ranges = compute_window_indices(self.targets)
        prec, recall = ts_precision_and_recall(self.targets, self.preds, alpha=0,
                                                anomaly_ranges=label_ranges,
                                                weighted_precision=True)
        f1 = (1 + 1**2) * prec * recall / (1**2 * prec + recall + epsilon) 
        return f1, prec, recall

def return_scores(evaluator):
    result_f1, prec, rec = evaluator.eval_naive_f1()
    result_f1, prec, rec = round(result_f1, 5), round(prec, 5),round(rec, 5)
    
    result_f1_pak, pre_pak, rec_pak = evaluator.eval_pak_auc()
    result_f1_pak, pre_pak, rec_pak = round(result_f1_pak, 5), round(pre_pak, 5),round(rec_pak, 5)
    
    result_f1_comp, pre_comp, rec_comp = evaluator.eval_f1_composite()
    result_f1_comp, pre_comp, rec_comp = round(result_f1_comp, 5), round(pre_comp, 5),round(rec_comp, 5)
    
    result_f1_range, pre_range, rec_range = evaluator.eval_f1_range()
    result_f1_range, pre_range, rec_range = round(result_f1_range, 5), round(pre_range, 5),round(rec_range, 5)
    
    return result_f1, prec, rec, result_f1_pak, pre_pak, rec_pak, result_f1_comp, pre_comp, rec_comp, result_f1_range, pre_range, rec_range

In [ ]:
y_preds = np.asarray(y_preds, dtype=int)
y_targets = np.asarray(y_targets, dtype=int)

evaluator = ano_evaluator(y_preds, y_targets)

f1, precision, recall, f1_k, precision_k, recall_k, f1_c, precision_c, recall_c, f1_r, precision_r, recall_r = return_scores(evaluator)
print("Point-wise F1:", f1)
print("Point-wise Precision:", precision)
print("Point-wise Recall:", recall)

print("F1-k (PAK):", f1_k)
print("Precision-k:", precision_k)
print("Recall-k:", recall_k)

print("F1-c (Composite):", f1_c)
print("Precision-c:", precision_c)
print("Recall-c:", recall_c)

print("F1-r (Range):", f1_r)
print("Precision-r:", precision_r)
print("Recall-r:", recall_r)